In [48]:
import random
import numpy as np

class card:

    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
    
    def __eq__ (self, other):
        return (self.value == other.value)

    def __ne__ (self, other):
        return (self.value != other.value)

    def __lt__ (self, other):
        return (self.value < other.value)

    def __le__ (self, other):
        return (self.value <= other.value)

    def __gt__ (self, other):
        return (self.value > other.value)

    def __ge__ (self, other):
        return (self.value >= other.value)
    
    def __add__ (self, int):
        return card(self.suit, self.value+1)
    
    def __sub__ (self, int):
        return card(self.suit, self.value-1)
    
    def __repr__(self):
        return f'{self.values[self.value]} of {self.suit}'

class deck:
   
    def __init__(self, randomize=True):
        self.suits = {"D":"Diamonds", "C":"Clubs", "H": "Hearts", "S":"Spades"}
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
        self.cards = np.array([])

        self.shuffle_deck(randomize)
    
    def shuffle_deck(self, randomize=True):
        self.cards = np.array([])
        for value in self.values.keys():
            for suit in self.suits.keys():
                self.cards = np.append(self.cards, card(suit, value))
        
        if randomize:
            np.random.shuffle(self.cards)

        return self.cards
    
    def __repr__(self):
        return f"{self.cards}"

    def draw(self):
        drawn_card = self.cards[0]
        self.cards = self.cards[1:]
        return drawn_card

class player:

    def __init__(self):
        self.hand = np.array([])
        self.money = 0
        self.status = True
        self.role = 4
        self.action = False
        self.high = None
        # {1: Dealer, 2: Little Blind, 3: Big Blind, 4: Nothing}
    
    def call(self, deck):
        self.money -= deck.call_amt

    def fold(self):
        self.status = False

    def raise_money(self, board, raise_amt):
        board.call_amt = raise_amt
        self.money -= board.call_amt


class board:

    def __init__(self, deck, blind_amt=10, players=None, buy_in=1000, call_amt=0):
        self.cards = np.array([])
        self.pot = 0
        self.buy_in = buy_in
        self.call_amt = 0
        self.blind = blind_amt
        self.players = players
        self.deck = deck
    
    def show(self):
        if len(self.cards) == 0:
            num = 3
        else:
            num = 1    
        for card in range(num):
            self.cards = np.append(self.cards, self.deck.draw())
        return self.cards
    
    def deal_hands(self):
        for player in self.players:
            player.hand = np.append(player.hand, [self.deck.draw(), self.deck.draw()])
    
    def reset(self):
        for player in self.players:
            player.hand = []
        self.deck.shuffle_deck()
        self.cards = []
    
    def card_sort(self, cards):
        values_order = np.argsort(cards[0,:].astype(int))[::-1]
        suit_order = np.argsort(cards)[1,:]
        
        return cards, values_order, suit_order
    
    def player_board(self, player):
        card_list = np.append(self.cards, player.hand)
        string_list = np.array([[card.value for card in card_list],
                                [card.suit for card in card_list]])
        return string_list

    def isStraight(self, cards):
        cards, values_order, _ = self.card_sort(cards)
       
    
    def isStraight2(self, cards):
        cards, values_order, _ = self.card_sort(cards)

        in_row = cards[values_order][:-1] == cards[values_order][1:] + 1

    def isFlush(self, cards):
        cards, _, suits_order = self.card_sort(cards)
        
        same = 0
        for num in range(len(cards)-1):
            if cards[suits_order][num].suit == cards[suits_order][num+1].suit:
                same += 1
                if same >= 4:
                    return cards[suits_order][num-3:num+2]
            else:
                same = 0

    def isStraightFlush(self, cards):
        cards, values_order, suits_order = self.card_sort(cards)

        cards_in_order = np.array([])
        for num in range(len(cards)-1):
            if cards[values_order][num] == cards[values_order][num+1]+1:
                cards_in_order = np.append(cards_in_order, cards[values_order][num])
                print(cards_in_order)
                if np.size(cards_in_order) >= 4:
                    # Adds remaining cards if they are still consecutive
                    while num < len(cards)-1 and cards[values_order][num] == cards[values_order][num+1]+1 :
                        cards_in_order = np.append(cards_in_order, cards[values_order][num+1])
                        num += 1
                    break           
            else:
                cards_in_order = np.array([])
        return self.isFlush(cards_in_order)
    
    def isRoyalFlush(self, cards):
        if self.isStraightFlush(cards)[0].value == 14:
            return self.isStraightFlush(cards)

    def isFour(self, player):
        card_values, _ = self.player_board(player)
        high_single = 0
        same_count = 0

        for n in range(len(card_values)-1):
            if card_values[n] == card_values[n+1]:
                same_count += 1
                if same_count == 3:
                    #if four of a kind is the greatest four in array, the fifth in array is the high single
                    if high_single == 0:
                        high_single = card_values[n+2]
                    return True, (card_values[n], high_single)
            elif card_values[n] > high_single:
                high_single = card_values[n]
                same_count = 0
            else:
                same_count = 0
        return False, None
    
    def isTriple(self, player):
        card_values, _ = self.player_board(player)
        same_count = 0
        
        for n in range(len(card_values)-1):
            if card_values[n] == card_values[n+1]:
                same_count += 1
                if same_count == 2:
                    return True, card_values[n]
            else:
                same_count = 0
        return False, None
    
    def isDouble(self, player):
        card_values, _ = self.player_board(player)
        doubles = []
        
        for n in range(len(card_values)-1):
            if card_values[n] == card_values[n+1]:
                if same_count == 2:
                    return True, card_values[n]
            else:
                same_count = 0
        return False, None


In [49]:
player1 = player()
play_deck = deck()
_board = board(play_deck, players=[player1])

In [55]:
_board.show()

array([7 of C, King of H, Jack of H, 7 of S, 4 of D, Ace of H, 9 of S,
       Queen of S], dtype=object)

In [56]:
_board.player_board(player1)

array([['7', '13', '11', '7', '4', '14', '9', '12'],
       ['C', 'H', 'H', 'S', 'D', 'H', 'S', 'S']], dtype='<U21')

In [57]:
c, v, s = _board.card_sort(_board.player_board(player1))

In [81]:
a = np.unique(c[0,v].astype(int))

In [83]:
a[1:] == a[:-1] + 1

array([False, False, False,  True,  True,  True])

array(['C', 'D', 'H', 'H', 'H', 'S', 'S', 'S'], dtype='<U21')